In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys
path = '/content/drive/My Drive/Final Project'
sys.path.append(path)

%cd $path

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/18oPzXa_o2Y_k8Bcbz3FNVsqteJ0Pt29Y/Final Project


In [ ]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from google.colab import drive
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import random

In [ ]:
cleaned_tweets = pd.read_csv("data/cleaned_elon.csv").drop(["Unnamed: 0"], axis=1)
cleaned_tweets.head()

,Tweet
0,Cowboy riding the rocket no problemo
1,Single camera view of the 40 meter rocket hover
2,Then we took him for a ride: 40 meter hover fl...
3,To provide a little perspective on the size of...
4,Very much agree with this great review of Chas...


In [ ]:
len(cleaned_tweets)

12622

In [ ]:
# Change this constant to determine the number of randomly sampled (without replacement) tweets we use from the dataset
NUM_TWEETS = 500

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

tweets = random.sample(list(cleaned_tweets["Tweet"]), NUM_TWEETS)
vectorizer = CountVectorizer()
## This one hot encodes the words and records all the corresponding words to their index in vectorizer.get_feature_names_out()
X = vectorizer.fit_transform(tweets)
len(vectorizer.get_feature_names_out())

2333

In [ ]:
import random
feature_names = vectorizer.get_feature_names_out()
print(f"Random feature/word: {feature_names[random.randint(0, len(feature_names))]}")

Random feature/word: market


In [ ]:
randind = random.randint(0, len(tweets))
words = tweets[randind].split(" ")
print(words)
print(f"Length of tweet in words: {len(words)}")
print(X.toarray()[randind].sum())

['Without', 'the', 'removal', 'fee,', 'Tesla', 'Solar', 'is', 'unequivocally', 'a', 'guaranteed,', 'instant', 'money', 'printer,', 'producing', '$300', 'to', '$1000', 'per', 'year', '(in', 'after', 'tax', 'income!)']
Length of tweet in words: 23
22


In [ ]:
print(len(tweets))

500


In [ ]:
%pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

nltk.download('stopwords')
en_stopws = stopwords.words('english')

corpus = " ".join(tweets)
corpus = corpus.lower()
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(corpus)
tokens = [token for token in tokens if token not in en_stopws]
fdist = FreqDist(tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


The following shows the 100 most common words. This gives us a preview of the types of words that appear most frequently and thus will be likely generated by the model.

In [ ]:
print(fdist.most_common(100))

[('tesla', 102), ('amp', 70), ('model', 50), ('rocket', 25), ('first', 23), ('falcon', 22), ('new', 22), ('like', 21), ('launch', 20), ('good', 20), ('car', 19), ('3', 18), ('time', 17), ('w', 16), ('spacex', 16), ('x', 16), ('space', 14), ('9', 14), ('long', 14), ('dragon', 14), ('mars', 14), ('today', 13), ('called', 13), ('team', 12), ('year', 12), ('test', 12), ('us', 12), ('solar', 12), ('next', 11), ('love', 11), ('landing', 11), ('real', 11), ('autopilot', 11), ('much', 11), ('station', 11), ('people', 11), ('company', 10), ('1', 10), ('california', 10), ('coming', 10), ('soon', 10), ('awesome', 10), ('day', 10), ('review', 10), ('need', 10), ('cars', 10), ('true', 10), ('also', 10), ('video', 9), ('article', 9), ('way', 9), ('great', 9), ('world', 9), ('many', 9), ('would', 9), ('flight', 9), ('btw', 9), ('show', 9), ('2', 9), ('worth', 9), ('takes', 8), ('high', 8), ('know', 8), ('right', 8), ('ship', 8), ('tank', 8), ('stage', 8), ('want', 8), ('la', 8), ('still', 8), ('almos

Let's clean some of the data. We removed most of the emojis in the preprocessing with a regex, but there are some still missing. We will remove them here.

In [ ]:
chars = sorted(list(set(''.join(tweets))))
print('total chars:', len(chars))

total chars: 90


In [ ]:
print(chars)

[' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', 'ō', '–', '’', '“', '”', '…', '🤘', '🤣']


In [ ]:
# to_exclude = ['🤔', '🤖', '🤗', '🤘', '🤣', '🤯', '🤹', '🥁', '🥇', '🥜', '🥵', '\U0001f979', '🦊', '🧀', '🧁', '🧐', '🧙', '🧛', '🧠', '🧦', '🧨', '🧲', '🩸']
# for i in range(len(tweets)):
#   for char in to_exclude:
#     if char in tweets[i]:
#       tweets[i] = tweets[i].replace(char, "")

In [ ]:
valid_characters = [' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [ ]:
# We want to limit the tweets to alphanumeric characters and general punctuation
for i in range(len(tweets)):
  for char in valid_characters:
    if char not in tweets[i]:
      tweets[i] = tweets[i].replace(char, "")

In [ ]:
chars = sorted(list(set(''.join(tweets))))
print('total chars:', len(chars))
num_chars = len(chars)

total chars: 90


In [ ]:
corpus = " ".join(tweets)

In [ ]:
# create idx2char dict and char2idx dict
char2idx = dict((c, i) for i, c in enumerate(chars))
idx2char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 32
max_seq_length = 40

In [ ]:
# This allows for reproducability of the training.
tf.compat.v1.reset_default_graph()
tf.random.set_seed(42)

In [ ]:
sentences = []
next_chars = []
# sliding window of shifted over sequences
for i in range(0, len(corpus)-max_seq_length):
  seq_in = corpus[i:i+max_seq_length]
  seq_out = corpus[i+max_seq_length]
  sentences.append([char2idx[i] for i in seq_in])
  next_chars.append(char2idx[seq_out])

n_sentences = len(sentences)
print(n_sentences)

43227


In [ ]:
# spread out sequences to form one long array of characters
X = [ch for sentence in sentences for ch in sentence]
print(len(X))

1729080


In [ ]:
X = keras.utils.to_categorical(X, num_classes=num_chars)

In [ ]:
print(X.shape)

(1729080, 90)


In [ ]:

X = np.reshape(X, (n_sentences, max_seq_length, len(X[0])))
print(X.shape)

# 67035 entries with each entry being 40 by 93
# 40 being sequence length and 93 being size of each one hot encoded vector (vocabulary)

y = keras.utils.to_categorical(next_chars, num_classes=len(chars))
y.shape
     

(43227, 40, 90)


(43227, 90)

The sequence length is determined. The way the model is trained is a where X=(40 characters of text) and y=(The next character). So the model is generating the character with the highest probability given the context of the previous 40.

In [ ]:

# define tf data and batch accordingly
# train val split
data_lstm = tf.data.Dataset.from_tensor_slices((X, y))
data_lstm = data_lstm.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

TRAIN_SIZE = int(0.85 * (n_sentences//BATCH_SIZE))
data_lstm = data_lstm.shuffle(buffer_size= BUFFER_SIZE, reshuffle_each_iteration=False)
train_lstm = data_lstm.take(TRAIN_SIZE)
val_lstm = data_lstm.skip(TRAIN_SIZE)

In [ ]:
train_lstm = train_lstm.cache()

val_lstm = val_lstm.cache()

In [ ]:
print(tf.compat.v1.data.get_output_shapes(data_lstm))

(TensorShape([32, 40, 90]), TensorShape([32, 90]))


In [ ]:
def build_lstm(rnn_units, dropout, batch_size):
  lstm = Sequential()
  lstm.add(layers.LSTM(rnn_units, return_sequences=True, batch_input_shape=(batch_size, max_seq_length, num_chars)))
  lstm.add(layers.Dropout(dropout))
  lstm.add(layers.LSTM(rnn_units))
  lstm.add(layers.Dropout(dropout))
  lstm.add(layers.Dense(num_chars, activation='softmax'))

  return lstm

lstm = build_lstm(1024, 0.1, BATCH_SIZE)
lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (32, 40, 1024)            4567040   
                                                                 
 dropout (Dropout)           (32, 40, 1024)            0         
                                                                 
 lstm_1 (LSTM)               (32, 1024)                8392704   
                                                                 
 dropout_1 (Dropout)         (32, 1024)                0         
                                                                 
 dense (Dense)               (32, 90)                  92250     
                                                                 
Total params: 13,051,994
Trainable params: 13,051,994
Non-trainable params: 0
_________________________________________________________________


In [ ]:
epochs = 10
# def perplexity(labels, preds):
#     return keras.backend.exp(tf.keras.losses.categorical_crossentropy(labels, preds))

# cc takes one hot encoded
def loss(labels, logits):
  return tf.keras.losses.categorical_crossentropy(labels, logits)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4, epsilon=1e-8)
  
lstm.compile(optimizer=optimizer, loss=loss) 

history_lstm = lstm.fit(train_lstm, epochs=epochs, validation_data=val_lstm, verbose=2)

Epoch 1/10
1147/1147 - 60s - loss: 2.9062 - val_loss: 2.5685 - 60s/epoch - 52ms/step
Epoch 2/10
1147/1147 - 49s - loss: 2.4602 - val_loss: 2.3103 - 49s/epoch - 43ms/step
Epoch 3/10
1147/1147 - 49s - loss: 2.2781 - val_loss: 2.1325 - 49s/epoch - 43ms/step
Epoch 4/10
1147/1147 - 50s - loss: 2.1124 - val_loss: 1.9675 - 50s/epoch - 44ms/step
Epoch 5/10
1147/1147 - 49s - loss: 1.9291 - val_loss: 1.8086 - 49s/epoch - 43ms/step
Epoch 6/10
1147/1147 - 50s - loss: 1.7001 - val_loss: 1.6559 - 50s/epoch - 44ms/step
Epoch 7/10
1147/1147 - 49s - loss: 1.4073 - val_loss: 1.5415 - 49s/epoch - 43ms/step
Epoch 8/10
1147/1147 - 49s - loss: 1.0796 - val_loss: 1.4353 - 49s/epoch - 43ms/step
Epoch 9/10
1147/1147 - 50s - loss: 0.7755 - val_loss: 1.3538 - 50s/epoch - 43ms/step
Epoch 10/10
1147/1147 - 50s - loss: 0.5452 - val_loss: 1.2724 - 50s/epoch - 43ms/step


In [ ]:
lstm.save_weights('lstm_weights_10_epoch.hdf5')

In [ ]:
# build model of batch 1 for evaluation
lstm1 = build_lstm(1024, 0.2, 1)
lstm1.load_weights('lstm_weights_10_epoch.hdf5')

In [ ]:
# sourced from keras tutorial
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:

# generating w/ random start tweet
start_index = random.randint(0, len(tweets))
for temperature in np.arange(0.25, 1.25, 0.25):
  generated = ''
  sentence = "We plan on releasing a new fleet of Tesl"#tweets[start_index][:max_seq_length]
  if temperature == 0.25:
    print('generating with: ' + sentence)
  generated += sentence
  print(temperature)
  for i in range(100):
    test = [char2idx[i] for i in sentence]
    test = keras.utils.to_categorical(test, num_classes=num_chars)
    test = tf.expand_dims(test, 0)
    preds = lstm1.predict(test, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_char = idx2char[next_index]
    sentence = sentence[1:] + next_char
    generated += next_char
  print(f"Temperature: {temperature}")
  print(generated)
     

generating with: We plan on releasing a new fleet of Tesl
0.25
Temperature: 0.25
We plan on releasing a new fleet of Tesla is anceally of not searce computies of Falcon book most plodictulating in the word … Test to be la
0.5
Temperature: 0.5
We plan on releasing a new fleet of Tesla is also loget a campond a complece to production a cal on the launch pod Sharkess arvitheraling wo
0.75
Temperature: 0.75
We plan on releasing a new fleet of Tesla is ancechould bowehtatedrw in Sepace &omp; govilatesy to beal. Model S ans commuch in the pad &amp
1.0
Temperature: 1.0
We plan on releasing a new fleet of Tesla deploded only Scancentises. A cripizats ramp; busigitar. Drains only bake. Tesla good enginear sol
